In [2]:
!pip install konlpy
!pip install eunjeon
import pandas as pd
pd.set_option('mode.chained_assignment',  None) 
import numpy as np

import selenium
from selenium import webdriver
import time
import bs4
from bs4 import BeautifulSoup
import unicodedata
import math
import re
import nltk
import konlpy
import eunjeon
from eunjeon import Mecab
m = Mecab()
hannanum = Hannanum()
okt = Okt()

chrome_path = # 본인 컴퓨터에 저장된 크롬드라이버 저장 위치

In [ ]:
def crawlling_pol(number, ChromeD_path): # 크롤링 후 해당 페이지의 html 소스를 리턴하는 함수
    driver =webdriver.Chrome(ChromeD_path)
    driver.get("https://policy.nec.go.kr/")

    iframe = driver.find_element_by_id("indexIframe")  # iframe 때문에 그냥 크롤링 하려고 들면 절대 안됨..ㅠ
    driver.switch_to.frame(iframe)
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/a[1]/div').click() # fullXpath로 복사해야함

    time.sleep(2)

    #이거 역시 iframe 문제인거 같은데 일단 excute_script 하니까 됨 ;; -> https://wkdtjsgur100.github.io/selenium-does-not-work-to-click/
    element = driver.find_element_by_xpath("""//*[@id="dataDiv"]/div[%d]/ul/li/a[1]""" % number) # 각자 후보자 정책 목록 클릭 버튼 xpath로
    driver.execute_script("arguments[0].click();", element)

    time.sleep(2)

    # 셀레늄 탭전환 https://noodle-dev.tistory.com/45
    driver.switch_to_window(driver.window_handles[1])

    iframe = driver.find_element_by_id("indexIframe")
    driver.switch_to.frame(iframe)

    #content = driver.find_elements_by_class_name('arccrodion-content')
    html = driver.page_source
    return html

def soup_content(number,html, ChromeD_path):
    soup = BeautifulSoup(html, 'html.parser')
    content = soup.find_all('div','accordion-content-inner')
    content = str(content)
    content = content.split('<div class="accordion-content-inner">')
    cont_list = []
    for i in range(len(content)):
        if i == 0:
            pass
        else:
            content_rmv = re.sub('(<([^>]+)>)', '',content[i])
            new_str = content_rmv
            new_str = new_str.replace('\n\n\t','')
            new_str = new_str.replace('\n','')
            new_str = new_str.replace('                            ','')
            new_str = new_str.replace('○','')
            new_str = new_str.replace(' 목 표','')
            new_str = new_str.replace('  ','')
            new_str = new_str.replace('□','')
            new_str = new_str.replace('■','')
            cont_list.append(new_str)
    #메인 페이지 접속
    #driver = webdriver.Chrome('C:\\Users\\user\\Downloads\\chromedriver')
    driver =webdriver.Chrome(ChromeD_path)
    driver.get("https://policy.nec.go.kr/") 

    #정당정책 페이지 접속
    iframe = driver.find_element_by_id("indexIframe") 
    driver.switch_to.frame(iframe)
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/a[1]/div').click() 

    
    element = driver.find_element_by_xpath("""//*[@id="dataDiv"]/div[%d]/ul/li/a[1]""" % number) # 각자 후보자 정책 목록 클릭 버튼 xpath로
    driver.execute_script("arguments[0].click();", element)

    time.sleep(2)

    # 셀레늄 탭전환 https://noodle-dev.tistory.com/45
    driver.switch_to_window(driver.window_handles[1])

    iframe = driver.find_element_by_id("indexIframe")  # 그냥 크롤링 하려고 들면 절대 안됨
    driver.switch_to.frame(iframe)
    time.sleep(3)

    #소제목 텍스트 반복문으로 긁어오기 
    title_text = []
    for n in range(1,20,2) :
        title_path = '/html/body/div/div/div/div/div[{0}]/p[@class = "accordion-questions"]'.format(n)
        title = driver.find_element_by_xpath(title_path)
        title_text.append(title.text)
    title_text
        
    pols = pd.DataFrame()
    pols['타이틀'] = title_text
    pols['내용'] = cont_list
    return pols


In [3]:
chrome_path = chrome_path # 본인 컴퓨터의 크롬드라이버 저장 위치 입력
for i in range(1,4):
    print(i)
    html_n = crawlling_pol(i,chrome_path)
    globals()['pols_{}'.format(i)] = soup_content(i,html_n,chrome_path)

    
# 카테고리 칼럼 생성
for i in range(1,4):
    title = globals()['pols_{}'.format(i)]
    title = title['타이틀']
    category_list= []
    for k in range(len(title)):
        detail = title[k]
        left = detail.find('[')
        right = detail.find(']')
        category = detail[left+1:right]
        category_list.append(category)
    globals()['pols_{}'.format(i)]['카테고리'] = category_list

1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning:

use driver.switch_to.window instead

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: DeprecationWarning:

use driver.switch_to.window instead



2
3


In [7]:
# 후보별 공약 데이터셋 다운받기
for i in range(1,4):
     globals()['pols_{}'.format(i)].to_excel('{}번_후보공약.xlsx'.format(i))